In [0]:
# import moduels
import requests # for crawling
from bs4 import BeautifulSoup # for crawling
import pandas as pd # for handle DataFrame
import numpy as np # for handle DataFrame
import os # for handle local environment
import re # for handel str()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class BeerPictureCrawling(object):
    beers = list() # 전역변수 beers
    
    def __init__(self): # instance 변수 사용 안함 
        pass
    
    # 1. 다운 받을 맥주 이름 불러오고 list로 변경
    def To_list(self, column_name):
        self.column_name = column_name
        
        Beer_names_df = pd.read_csv("/content/drive/My Drive/beer_name.csv", encoding='utf-8')
        beer_names = list(Beer_names_df[column_name])
        beer_names.sort() 

        # beers에 담기 
        for beer in beer_names:
            beer = beer + " beer" # beer를 넣어야 검색 효율성이 증대

            # 특수문자를 제거함으로써 검색 효율성 증대 + 오류 방지
            parsed_beer = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', beer)

            # 구글은 + 문자를 인식하므로, 공백을 + 로 대체 
            parsed_beer = parsed_beer.replace(" ","+")
            
            # beers 리스트에 append 
            BeerPictureCrawling.beers.append(parsed_beer)
            
        return BeerPictureCrawling.beers
            
    # 2. 구글 이미지 검색으로 requests 요청
    def BeerCrawling(self):
        count = 0 # 결과 수를 count
        error_count = 0 # 에러 수를 count
        error_list = list() # error가 뜬 맥주를 확인하기 위한 error list()
        
        ##############################################################################
        
        # 폴더 생성 code
        BASE_DIR = "/content/drive/My Drive"
        PICTURE_DIR = os.path.join(BASE_DIR, 'beer_pictures')

        # 만약 경로가 없으면, 새로운 폴더를 생성하겠습니다.
        if not os.path.exists(PICTURE_DIR):
            os.makedirs(PICTURE_DIR) 
        
        ##############################################################################
        
        # Crawling codes    
        for b in BeerPictureCrawling.beers:
            try:
                filename= b.replace("+"," ")[:-5] + ".png" # png 형식의 파일 
                print(filename)
                BASE_URL = "https://www.google.com/search"
                params = {
                    'tbm': 'isch',
                    'q': b,
                    'tbs': 'ic:specific,isc:white'
                }

                resp = requests.get(BASE_URL, params=params)
                # print(resp)
                # print(resp.url)
                
                soup = BeautifulSoup(resp.content, 'html.parser')
                # print(soup)

                beer_tag = str(soup.find_all('a')[40]) # 가져올 이미지 링크가 존재하는 tag
                # print(beer_tag)
                
                if "spell=1" in beer_tag: # "이것을 찾으셨나요?, 수정된 검색어에 대한 결과" tag
                    beer_tag = str(soup.find_all('a')[41])
                    if "nfpr=1" in beer_tag: # "다음 검색어로 대신 검색" tag
                        beer_tag = str(soup.find_all('a')[42])
                    else:
                        pass
                else:
                    pass

                beer_picture_url = beer_tag.split(" ")[-2].replace("src=","").replace('"',"")
                # print(beer_picture_url)

                img_resp = requests.get(beer_picture_url)
                # print(resp2)

                beers_picture_path = os.path.join(PICTURE_DIR, filename)
                with open(beers_picture_path, 'wb') as f:
                    f.write(img_resp.content) # 이미지 파일 쓰기

                count += 1

            except:
                error_list.append(b)
                error_count += 1
                pass

        # 문자열 추가 
        print("전체 맥주의 수: ", len(BeerPictureCrawling.beers)) # 전체 맥주 이름 list 
        print("\n수집된 맥주의 수: ", count) # 실제로 수집된 맥주 수 
        print("\n에러가 난 맥주의 수: ", error_count) # 에러가 발생한 맥주 수 
        print("\n0?", len(BeerPictureCrawling.beers) - (count + error_count)) # 0이 나와야 정상
        print("\n Error List: \n", error_list)

In [0]:
Test = BeerPictureCrawling()

In [11]:
Test.To_list('name')

['+100+beer',
 '1812+beer',
 '19+beer',
 '3+Monts+Amber+Ale+On+Lees+beer',
 '3+Monts+Flanders+Golden+Ale+beer',
 '4+Pines+Dry+Stout+beer',
 '90+Wee+Heavy+beer',
 'APA+beer',
 'Aaron+beer',
 'Aass+Bayer+Aass+Dark+beer',
 'Aass+Bock+beer',
 'Aass+Genuine+Pilsner+beer',
 'Aass+Juleøl+beer',
 'Abbaye+Des+Rocs+Grand+Cru+Belgian+Special+Brown+beer',
 'Abbaye+Des+Rocs+Brasserie+Des+Rocs+Brune+beer',
 'Abrahadabra+beer',
 'Abricot+Du+Fermier+beer',
 'Abt+12+beer',
 'Abt+12+Oak+Aged+beer',
 'Ace+Of+Spades+beer',
 'Achel+Trappist+Extra+beer',
 'Adnams+TallyHo+beer',
 'Aecht+Schlenkerla+Eiche+beer',
 'Aecht+Schlenkerla+Fastenbier+beer',
 'Aecht+Schlenkerla+Helles+Lagerbier+beer',
 'Aecht+Schlenkerla+Kräusen+beer',
 'Aecht+Schlenkerla+Rauchbier+Märzen+beer',
 'Aecht+Schlenkerla+Rauchbier+Urbock+beer',
 'Aecht+Schlenkerla+Rauchbier+Weizen+beer',
 'Agamemnon+beer',
 'Agent+Provocateur+beer',
 'Akevitt+beer',
 'AlIksir+beer',
 'Ale+Gâteau+ForêtNoire+beer',
 'Alpha+Pale+Ale+beer',
 'Altbier+beer',
 'A

In [12]:
Test.BeerCrawling()

 100.png
1812.png
19.png
3 Monts Amber Ale On Lees.png
3 Monts Flanders Golden Ale.png
4 Pines Dry Stout.png
90 Wee Heavy.png
APA.png
Aaron.png
Aass Bayer Aass Dark.png
Aass Bock.png
Aass Genuine Pilsner.png
Aass Juleøl.png
Abbaye Des Rocs Grand Cru Belgian Special Brown.png
Abbaye Des Rocs Brasserie Des Rocs Brune.png
Abrahadabra.png
Abricot Du Fermier.png
Abt 12.png
Abt 12 Oak Aged.png
Ace Of Spades.png
Achel Trappist Extra.png
Adnams TallyHo.png
Aecht Schlenkerla Eiche.png
Aecht Schlenkerla Fastenbier.png
Aecht Schlenkerla Helles Lagerbier.png
Aecht Schlenkerla Kräusen.png
Aecht Schlenkerla Rauchbier Märzen.png
Aecht Schlenkerla Rauchbier Urbock.png
Aecht Schlenkerla Rauchbier Weizen.png
Agamemnon.png
Agent Provocateur.png
Akevitt.png
AlIksir.png
Ale Gâteau ForêtNoire.png
Alpha Pale Ale.png
Altbier.png
Alter Ego.png
Amager  Grassroots  Shadow Pictures  Skyggebilleder.png
Amager  Jester King  Danish Metal.png
Amager  Mikkeller  Hr Frederiksens Væsel Brunch.png
Amager  Mikkeller  Hr F

In [1]:
len(['Aass+Bayer+Aass+Dark+beer',
 'Ale+Gâteau+ForêtNoire+beer',
 'Ayinger+UrWeisse+beer',
 'Beatification+beer',
 'Black+Damnation+II++Mocha+Bomb+beer',
 'Colorado+Appia+beer',
 'Cuvee+Delphine+beer',
 'Cuvée+Des+Jonquilles+beer',
 'Day+Of+The+Dead+Beer+Cerveza+De+Los+Muertos+Death+Becomes+You+beer', 
 'Day+Of+The+Dead+Beer+Cerveza+De+Los+Muertos+Pay+The+Ferryman+beer',
 'Dugges++Stillwater++Mango+Mango+Mango+beer',
 'Extra+beer',
 'Fuck+Art++Lets+Dance+beer',
 'God+Jul+beer',
 'Granát++BrouCzech+Dark+beer',
 'Hel++Verdoemenis+Misto+BA+beer',
 'Hirter+1270+beer',
 'Hirter+Morchl+beer',
 'Hitachino+Nest+Yuzu+Lager+beer',
 'Imperial+Brown+Ale+beer',
 'LAmalthée+beer',
 'Maelstrom+beer', 
 'Mooi++Meedogenloos+beer',
 'Ola+Dubh+Special+Reserve+16+beer',
 'Old+Chimneys+Good+King+Henry+Imperial+Stout+beer',
 'Stifter+Bier+Red+Ale+beer',
 'Sunday+Brunch+beer', 'Tak+Pale+Ale+beer', 'Tuatara+Helluva+beer'])

29

In [2]:
1148 + 28

1176